In [1]:
import subprocess
import time
import requests

DOCKER_IMAGE = "searxng/searxng"  # Beispiel: Docker Image von SearXNG
CONTAINER_NAME = "searxng_container"
SEARXNG_URL = "http://localhost:8080/search"  # URL von SearXNG



In [2]:
def start_docker():
    print("Starte den Docker-Container...")
    subprocess.run(["docker", "run", "-d", "--name", CONTAINER_NAME, "-p", "8080:8080", DOCKER_IMAGE])

    # Warte auf den Serverstart
    for _ in range(30):  # Maximal 30 Sekunden warten
        try:
            response = requests.get(SEARXNG_URL)
            if response.status_code == 200:
                print("SearXNG ist bereit!")
                return
        except requests.ConnectionError:
            pass
        
        time.sleep(1)

    print("Fehler: SearXNG hat nicht rechtzeitig gestartet!")


In [3]:
start_docker()

Starte den Docker-Container...
082889590c273ec9408232cd9612a84387e9640a7c9b852bb1c74d5a71816fba
SearXNG ist bereit!


In [20]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import quote

def searx_search(query, base_url='http://localhost:8080'):
    encoded_query = quote(query)
    url = f"{base_url}/search?q={encoded_query}&categories=general"
    
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Änderung: Suche nach Artikeln mit voller Klasse
        results = soup.find_all('article', class_='result result-default category-general')
        
        parsed_results = []
        for result in results:
            # Exakte Pfade zur Extraktion
            title_elem = result.find('h3').find('a')
            content_elem = result.find('p', class_='content')
            url_elem = result.find('a', class_='url_header')
            
            parsed_results.append({
                'title': title_elem.text.strip() if title_elem else 'Kein Titel',
                'link': url_elem['href'] if url_elem else '',
                'snippet': content_elem.text.strip() if content_elem else 'Keine Beschreibung'
            })
        
        return parsed_results
    
    except Exception as e:
        print(f"Fehler: {e}")
        return None

# Beispiel
results = searx_search("python programmierung")
for result in results:
    print(f"Titel: {result['title']}")
    print(f"Link: {result['link']}")
    print(f"Beschreibung: {result['snippet']}\n")

Titel: Python lernen - Python Kurs für Anfänger und Fortgeschrittene
Link: https://www.python-lernen.de
Beschreibung: Dieses Python Tutorial entsteht im Rahmen von Uni-Kursen und kann hier kostenlos genutzt werden. Python ist eine für Anfänger und Einsteiger sehr gut geeignete Programmiersprache, die später auch den Fortgeschrittenen und Profis alles bietet, was man sich beim Programmieren wünscht.

Titel: Python (Programmiersprache) - Wikipedia
Link: https://de.wikipedia.org/wiki/Python_(Programmiersprache)
Beschreibung: Python ist eine universell nutzbare, üblicherweise interpretierte, höhere Programmiersprache. Sie hat den Anspruch, einen gut lesbaren, knappen Programmierstil zu fördern. So werden beispielsweise Blöcke nicht durch geschweifte Klammern, sondern durch Einrückungen strukturiert. Python unterstützt mehrere Programmierparadigmen, z. B. die ...

Titel: Programmieren in Python - Universität Regensburg
Link: https://www.uni-regensburg.de/assets/physik/fakultaet/IT/Tutorials

In [4]:
# Docker stoppen
def stop_docker():
    print("Stoppe den Docker-Container...")
    subprocess.run(["docker", "stop", CONTAINER_NAME])
    subprocess.run(["docker", "rm", CONTAINER_NAME])


In [5]:
stop_docker()

Stoppe den Docker-Container...
searxng_container
searxng_container
